# To evaluate a decoherence curve from an underlying noise spectrum
Siddharth Dhomkar, Department of Physics, IIT Madras

**Based on**:

D.F. Wise, J.J.L. Morton, and S. Dhomkar, “Using deep learning to understand and mitigate the qubit noise environment”, PRX Quantum 2, 010316 (2021)



*   https://doi.org/10.1103/PRXQuantum.2.010316
*   http://dx.doi.org/10.1103/PhysRevApplied.18.024004




## Initialise

In [ ]:
# Colab
from google.colab import files
import os

# Other useful packages:
from plotly.subplots import make_subplots
import numpy as np
from plotly import graph_objs as go
from tqdm.notebook import trange
import pandas as pd
from scipy.interpolate import interp1d
from IPython.display import clear_output
from scipy.optimize import curve_fit
from scipy.interpolate import InterpolatedUnivariateSpline
import copy
import random

from scipy.optimize import Bounds, minimize, LinearConstraint

### Figure template

In [ ]:
fig_template = go.layout.Template()
fig_template.layout = {
    'template': 'simple_white+presentation',
    'autosize': False,
    'width': 800,
    'height': 600,
    # 'opacity': 0.2,
    'xaxis': {
        'title': 'Time (\u03BCs)',
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 2.5,
        'tickwidth': 2.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white',
        },
    'yaxis': {
        'title': 'Coherence',
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 2.5,
        'tickwidth': 2.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white'
        },
    'font':{'family':'mathjax',
            'size': 16,
            },
    'colorway': ["#d9ed92","#b5e48c","#99d98c","#76c893","#52b69a","#34a0a4","#168aad","#1a759f","#1e6091","#184e77"]
}

## Initialise

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# predicted_data_path = '/content/gdrive/My Drive/Research/ML_PROJECTS_ON_SUPERCONDUCTING_QUBITS/CNN/TRAINED_NETWORKS/'
predicted_data_path = '/content/gdrive/My Drive/ML_PROJECTS_ON_SUPERCONDUCTING_QUBITS/CNN/TRAINED_NETWORKS/'

In [ ]:
predicted_data = np.load(predicted_data_path+'predictions_Mar13.npz')
# predicted_data = np.load('/content/drive/MyDrive/ML_PROJECTS_ON_SUPERCONDUCTING_QUBITS/CNN/TRAINED_NETWORKS/predictions_Mar13.npz')

In [ ]:
predicted_data.files

['c_expt', 't_expt', 's_predict', 'w_predict']

## Functions to generate coherence curve from a given set of noise spectra

It is virtually impossible to extract the underlying noise spectrum from a decoherence curve. However, it is possible to obtain an unique coherence decay, if the precise functional form of the noise spectrum is known. This observation is utilized to generate the training datasets.

Qubit noise spectra, $S(\omega)$, can be of various functional forms including $1/f$-like, Lorentzian-like, double-Lorentzian-like etc. It is possible to simulate such noise spectra and corresponding decoherence curve, $C(t)$, can then be evaluated consequently using following relation:

$$- ln C(t) =  \int_{0}^{\infty}\frac{d\omega}{\pi}S(\omega)\frac{F(\omega t)}{\omega^{2}}$$

where, $F(\omega t)$ is the filter function associated with the decoupling protocol used for probing the decoherence dynamics.

(The detailed explanation of data generation process can be found in the manuscript.)

**Generate the filter function**

References pertaining to superconducting qubit noise spectra:

*   https://doi.org/10.1038/ncomms1856
*   https://www.nature.com/articles/ncomms12964
*   https://doi.org/10.1103/PhysRevApplied.18.044026
*   https://doi.org/10.1103/PhysRevResearch.3.013045
*   http://dx.doi.org/10.1103/PhysRevB.79.094520
*   http://dx.doi.org/10.1103/PhysRevB.77.174509
*   https://doi.org/10.1063/1.5089550


In [ ]:
# %%
# Create CPMG-like pulse timing array
def cpmgFilter(n, Tmax):
    tpi = np.empty([n])
    for i in range(n):
        tpi[i]= Tmax*(((i+1)-0.5)/n)
    return tpi


# %%
# Generate filter function for a given evolution time (CPMG)
def getFilter(n,w0,piLength,Tmax):

    tpi = cpmgFilter(n,Tmax)
    f = 0
    for i in range(n):
        f = ((-1)**(i+1))*(np.exp(1j*w0*tpi[i]))*np.cos((w0*piLength)/2) + f

    fFunc = (1/2)*((np.abs(1+((-1)**(n+1))*np.exp(1j*w0*Tmax)+2*f))**2)/(w0**2)
    return fFunc


### Evaluate the Coherence curves

In [ ]:
# %%
# Generate decoherence curve corresponding to a noise spectrum (input shape = variable1.size x w.size)
def getCoherence(S,w0,T0,n,piLength):
    steps = T0.size
    C_invert = np.empty([S.shape[0],steps,])
    for i in trange(steps):
        integ = getFilter(n,np.squeeze(w0),piLength,T0[i])*S/np.pi
        integ_ans = np.trapz(y=integ,x=np.squeeze(w0))
        C_invert[:,i] = np.exp(integ_ans)
    return C_invert


# %%
# Stretched-exponential function
def stretchExp(T0,T2,p,A):
    C = A*np.exp(-((T0/T2)**p))
    return C


# %%
# Extract T2, p, and amplitude by fitting the data with stretched-exponential function
def fit_stretchExp(C,T0):
    params = curve_fit(stretchExp, T0, C, bounds=([10e-6,0.9,0.995],[1000e-6,3.2,1.005]))
    T2, p, A = params[0]
    T2err, perr, Aerr = np.sqrt(np.diag(params[1]))
    return T2, p, A, T2err, perr, Aerr


# %%
# Fit multiple coherence curves to obtain values of T2 and p (stretching factor)
def get_fitPar(c_check,T_train):
  T2_check = []
  p_check = []
  for i in trange(c_check.shape[0]):
    T2_check0, p_check0, _, _, _, _ = fit_stretchExp(c_check[i,:],T_train)
    T2_check.append(T2_check0)
    p_check.append(p_check0)
  return np.round((np.array(T2_check)*1e6),1),np.round((np.array(p_check)),2)

# %%
# Moving average
def moving_average(x, width):
    return np.convolve(x, np.ones(width), 'valid') / width

## Optimal DD using Scipy

In [ ]:
# %%
# Generate filter function for arbitrary pulse sequence
def arbFilter(w0,piLength,tpi,Tmax):
  # print(tpi)
  n = tpi.size
  f = 0
  for i in range(n):
      f = ((-1)**(i+1))*(np.exp(1j*w0*tpi[i]))*np.cos((w0*piLength)/2) + f

  fFunc = (1/2)*((np.abs(1+((-1)**(n+1))*np.exp(1j*w0*Tmax)+2*f))**2)/(w0**2)
  return fFunc

# %%
# Generate decoherence curve corresponding to a noise spectrum
def optCoherence(S,w0,piLength,tpi,Tmax):
    integ = arbFilter(w0,piLength,tpi,Tmax)*S/np.pi
    integ_ans = np.trapz(y=integ,x=w0)
    C_invert = np.exp(integ_ans)
    return C_invert

In [ ]:
predicted_data.files

['c_expt', 't_expt', 's_predict', 'w_predict']

In [ ]:
c_in = predicted_data['c_expt']
s_in = predicted_data['s_predict']
w_in = predicted_data['w_predict']
T_in = predicted_data['t_expt']

In [ ]:
# n_plot = 10
# rand_set = np.random.randint(0,s_in.shape[0],(n_plot,))

In [ ]:
print(c_in.shape)
print(s_in.shape)
print(w_in.shape)
print(T_in.shape)

(16, 150)
(16, 1001)
(1001,)
(150,)


In [ ]:
T_in.min()

2.0479999999999997e-06

In [ ]:
fig = go.Figure()
for i in range(16):
  fig.add_scatter(x=1e-6*w_in/(2*np.pi),y=s_in[i,:],line=dict(width=2),opacity=0.8,name=i)

fig.update_layout(template=fig_template,  title = 'Noise Spectra',
                  width = 1000,
                  xaxis = dict(title='\N{greek small letter omega}/2\N{greek small letter pi} (MHz)',
                               type="log",
                              #  range=[5,50],
                               ),
                  yaxis = dict(title='S(\N{greek small letter omega})',
                              #  range=[-0.1e6,0.1e6],
                               type="log",

                               ),
                 )
fig

In [ ]:
c_test = getCoherence(np.reshape(s_extend,(1,s_extend.size)),w_extend,T_in,1,piLength)

  0%|          | 0/150 [00:00<?, ?it/s]

In [ ]:
c_test.shape

(1, 150)

In [ ]:
fig = go.Figure()
for i in range(16):
  fig.add_scatter(x=T_in*1e6,y=c_in[i,:],line=dict(width=2),opacity=0.8,name=i)
# fig.add_scatter(x=T_in*1e6,y=c_in[s_select,:],line=dict(width=2),opacity=0.8,
#                 name='Original',
#                 )
# fig.add_scatter(x=T_in*1e6,y=np.squeeze(c_test),line=dict(width=2),opacity=0.8,
#                 name='New',
#                 )
fig.update_layout(template=fig_template,  title = 'Coherence Curves',
                  width = 800,
                  xaxis = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,400]),
                  yaxis = dict(title='Coherence'),
                 )

In [ ]:
w_extra = 1001
w_low = np.flipud(np.linspace(10,w_in[-1]-10,w_extra))
s_low = 1e6*np.ones(w_extra)

In [ ]:
w_in.min()

1000.0

In [ ]:
# s_select = 15
# n = 2
# piLength = 48e-9
# Tmax = 60e-6

# w_extend = np.concatenate((w_in,w_low))
# s_extend = np.concatenate((s_in[s_select,:],s_low))

# def customDD(x):
#   c_val = -optCoherence(s_extend,w_extend,piLength,x,Tmax)
#   return c_val

In [ ]:
data_points = 20
n_plots = 15
C_DD = np.zeros((n_plots,data_points))
C_CPMG = np.zeros((n_plots,data_points))
n = 8
piLength = 48e-9
Tmax = np.linspace(5,300,data_points)*1e-6
w_extend = np.concatenate((w_in,w_low))
for k in trange (n_plots):

  s_extend = np.concatenate((s_in[k,:],s_low))

  lower_bounds = np.zeros(n)
  for i in range (n):
    lower_bounds[i] = (1/2+i)*piLength

  coeff_matrix = -np.diag(np.ones(n))
  for i in range(n-1):
    coeff_matrix[i, i+1] = 1

  M = coeff_matrix[:-1]
  lower_B = piLength*np.ones(n-1)


  for j in range(data_points):
    upper_bounds = np.zeros(n)
    for i in range (n):
      upper_bounds[i] = Tmax[j]-(n-(2*i+1)/2)*piLength

    bounds  = Bounds(lower_bounds,upper_bounds)

    upper_B = (Tmax[j]-(n-1)*piLength)*np.ones(n-1)
    linear_constraint = LinearConstraint(M,lower_B,upper_B)

    x0 = cpmgFilter(n,Tmax[j])
    def customDD(x):
      c_val = -optCoherence(s_extend,w_extend,piLength,x,Tmax[j])
      return c_val

    res = minimize(customDD, x0, method='SLSQP',
               constraints=[linear_constraint], options={'ftol': 1e-9, 'disp': True},
               bounds=bounds)
    C_DD[k,j] = -customDD(res.x)
    C_CPMG[k,j] = -customDD(x0)

In [ ]:
fig = go.Figure()
for i in range(n_plots):
  fig.add_scatter(x=Tmax,y=C_DD[i,:], name = F'C_DD_{i}')
  fig.add_scatter(x=Tmax,y=C_CPMG[i,:], name = F'C_CPMG_{i}')
fig.update_layout(template=fig_template,  title = 'Filter Functions',
                  width = 1000,
                  # xaxis = dict(title='\N{greek small letter omega} (MHz)',range=[-6,1],type="log",),
                  xaxis = dict(title = 't'),
                  yaxis = dict(title='C(t)'),
                 )
fig.show()

In [ ]:
 np.linspace(4,32,8)

array([ 4.,  8., 12., 16., 20., 24., 28., 32.])

In [ ]:
np.linspace(2,4,n_pi).astype(int)

array([2, 4])

In [ ]:
s_select = 2
n_pi = 8
n = np.linspace(4,32,n_pi).astype(int)
# n  = np.linspace(2,4,n_pi).astype(int)

piLength = 48e-9
data_points = 100
Tmax = np.linspace(5,700,data_points)*1e-6
w_extend = np.concatenate((w_in,w_low))
s_extend = np.concatenate((s_in[s_select,:],s_low))
C_vals_new = np.zeros((n_pi,data_points)) # Coherence for optimal DD
C_vals = np.zeros((n_pi,data_points))     # Coherence for CPMG
T_vals_new = {}
T_vals = {}
for k in trange(n_pi):

  lower_bounds = np.zeros(n[k])
  for i in range (n[k]):
    lower_bounds[i] = (1/2+i)*piLength

  coeff_matrix = -np.diag(np.ones(n[k]))
  for i in range(n[k]-1):
    coeff_matrix[i, i+1] = 1

  M = coeff_matrix[:-1]
  lower_B = piLength*np.ones(n[k]-1)

  for j in range(data_points):
    upper_bounds = np.zeros(n[k])
    for i in range (n[k]):
      upper_bounds[i] = Tmax[j]-(n[k]-(2*i+1)/2)*piLength

    bounds  = Bounds(lower_bounds,upper_bounds)

    upper_B = (Tmax[j]-(n[k]-1)*piLength)*np.ones(n[k]-1)
    linear_constraint = LinearConstraint(M,lower_B,upper_B)

    x0 = cpmgFilter(n[k],Tmax[j])
    def customDD(x):
      c_val = -optCoherence(s_extend,w_extend,piLength,x,Tmax[j])
      return c_val

    res = minimize(customDD, x0, method='SLSQP',
               constraints=[linear_constraint], options={'ftol': 1e-9, 'disp': True},
               bounds=bounds)
    # T_vals_new[F'T_vals_new_{n[k]}'] = res.x
    # T_vals[F'T_vals_{n[k]}'] = x0
    T_vals_new.update({F'T_vals_{n[k]}_{j}':res.x})
    T_vals.update({F'T_vals_{n[k]}_{j}':x0})
    C_vals_new[k,j] = -customDD(res.x)
    C_vals[k,j] = -customDD(x0)


  0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.9987386471145621
            Iterations: 4
            Function evaluations: 39
            Gradient evaluations: 4
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.9851324365507113
            Iterations: 7
            Function evaluations: 61
            Gradient evaluations: 7
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.9589725994838489
            Iterations: 9
            Function evaluations: 74
            Gradient evaluations: 9
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.9269558117154134
            Iterations: 7
            Function evaluations: 58
            Gradient evaluations: 7


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.892245140221713
            Iterations: 7
            Function evaluations: 58
            Gradient evaluations: 7
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.8566797247520181
            Iterations: 15
            Function evaluations: 119
            Gradient evaluations: 15
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.8212297947775188
            Iterations: 10
            Function evaluations: 79
            Gradient evaluations: 10


...

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.03090454453939888
            Iterations: 73
            Function evaluations: 2622
            Gradient evaluations: 73


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.029807839949812873
            Iterations: 62
            Function evaluations: 2241
            Gradient evaluations: 62


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Iteration limit reached    (Exit mode 9)
            Current function value: -0.028316067059009256
            Iterations: 100
            Function evaluations: 3518
            Gradient evaluations: 100


In [ ]:
np.savez('/content/gdrive/MyDrive/C_&_T_vals.npz',C_vals_new = C_vals_new,C_vals = C_vals,T_vals_new = T_vals_new,T_vals = T_vals)

In [ ]:
n_pi = 8
np.linspace(4,32,n_pi).astype(int)

array([ 4,  8, 12, 16, 20, 24, 28, 32])

In [ ]:
fig = go.Figure()
# fig.add_scatter(x=Tmax,y=C_vals_new, name = F'C_DD_{n}')
# fig.add_scatter(x=Tmax,y=C_vals, name = F'C_CPMG_{n}')
# fig.add_scatter(x=Tmax,y=C_vals32, name = F'C_CPMG32')

# fig.add_scatter(x=Tmax,y=100*(C_vals_new/C_vals - 1), name = F'C_CPMG_{n}')
# fig.add_scatter(x=Tmax,y= 100*(C_vals_new - C_vals), name = F'C_CPMG_{n}')
for i in range(8):
  fig.add_scatter(x=Tmax,y=100*(C_vals_new[i,:]/C_vals[i,:] - 1), name = F'C_CPMG_{n[i]}')

fig.update_layout(template=fig_template,  title = 'Comparision_Plots',
                  width = 1000,
                  # xaxis = dict(title='\N{greek small letter omega} (MHz)',range=[-6,1],type="log",),
                  xaxis = dict(title = 't'),
                  # yaxis = dict(title='C(t)'),
                  yaxis = dict(title='Coherence Improvement (%)',
                              #  type="log",
                              #  range=[-6,2],
                               ),
                 )
fig.show()

In [ ]:
s_select = 2
n_pi = 2
n  = np.linspace(2,4,n_pi).astype(int)
piLength = 48e-9
data_points = 10
Tmax = np.linspace(5,700,data_points)*1e-6
w_extend = np.concatenate((w_in,w_low))
s_extend = np.concatenate((s_in[s_select,:],s_low))
C_vals_new = np.zeros((n_pi,data_points)) # Coherence for optimal DD
C_vals = np.zeros((n_pi,data_points))     # Coherence for CPMG
T_vals_new = []
T_vals = []
for k in trange(n_pi):
  lower_bounds = np.zeros(n[k])
  for i in range (n[k]):
    lower_bounds[i] = (1/2+i)*piLength

  coeff_matrix = -np.diag(np.ones(n[k]))
  for i in range(n[k]-1):
    coeff_matrix[i, i+1] = 1

  M = coeff_matrix[:-1]
  lower_B = piLength*np.ones(n[k]-1)

  for j in range(data_points):
    upper_bounds = np.zeros(n[k])
    for i in range (n[k]):
      upper_bounds[i] = Tmax[j]-(n[k]-(2*i+1)/2)*piLength

    bounds  = Bounds(lower_bounds,upper_bounds)

    upper_B = (Tmax[j]-(n[k]-1)*piLength)*np.ones(n[k]-1)
    linear_constraint = LinearConstraint(M,lower_B,upper_B)

    x0 = cpmgFilter(n[k],Tmax[j])
    def customDD(x):
      c_val = -optCoherence(s_extend,w_extend,piLength,x,Tmax[j])
      return c_val

    res = minimize(customDD, x0, method='SLSQP',
               constraints=[linear_constraint], options={'ftol': 1e-9, 'disp': True},
               bounds=bounds)
    C_vals_new[k,j] = -customDD(res.x)
    C_vals[k,j] = -customDD(x0)
    T_vals_new.append(res.x)
    T_vals.append(x0)

# T_vals[:n[k],j] = res.x

  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.995760658095736
            Iterations: 6
            Function evaluations: 35
            Gradient evaluations: 6
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.6385871360748662
            Iterations: 10
            Function evaluations: 52
            Gradient evaluations: 9
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.40366204249159215
            Iterations: 5
            Function evaluations: 40
            Gradient evaluations: 5
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.2552677132534885
            Iterations: 6
            Function evaluations: 37
            Gradient evaluations: 6


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x wer

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.16149497592263848
            Iterations: 6
            Function evaluations: 37
            Gradient evaluations: 6
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.10215999629099959
            Iterations: 6
            Function evaluations: 36
            Gradient evaluations: 6
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.06462785924835807
            Iterations: 4
            Function evaluations: 28
            Gradient evaluations: 4
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.0408835634889942
            Iterations: 4
            Function evaluations: 32
            Gradient evaluations: 4
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.025851950839685563
            Iterations: 4
            Function ev

...

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.166542009960773
            Iterations: 15
            Function evaluations: 137
            Gradient evaluations: 15
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.1053605949057422
            Iterations: 8
            Function evaluations: 73
            Gradient evaluations: 8
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.06662536802535315
            Iterations: 2
            Function evaluations: 25
            Gradient evaluations: 2


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.042154477853606286
            Iterations: 5
            Function evaluations: 53
            Gradient evaluations: 5
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.02666472817404742
            Iterations: 6
            Function evaluations: 58
            Gradient evaluations: 6


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.01687453408605881
            Iterations: 7
            Function evaluations: 64
            Gradient evaluations: 7


In [ ]:
T_vals_new

[array([1.38556370e-06, 3.60045324e-06]),
 array([1.20649946e-05, 5.31006068e-05]),
 array([3.92645055e-05, 1.18974139e-04]),
 array([5.81202984e-05, 1.76419226e-04]),
 array([7.72343622e-05, 2.34052430e-04]),
 array([9.63693574e-05, 2.91635170e-04]),
 array([0.00011739, 0.00035094]),
 array([0.00013684, 0.00040868]),
 array([0.0001563 , 0.00046623]),
 array([0.00017572, 0.00052423]),
 array([6.62325150e-07, 1.85939292e-06, 3.05705871e-06, 4.27450761e-06]),
 array([7.22797298e-06, 2.16907635e-05, 4.81419093e-05, 7.49942496e-05]),
 array([6.16932621e-06, 4.59629871e-05, 1.13331847e-04, 1.53287289e-04]),
 array([3.33577381e-05, 7.54175441e-05, 1.18154664e-04, 1.94261932e-04]),
 array([3.80167238e-05, 1.16082180e-04, 1.94337449e-04, 2.73202819e-04]),
 array([4.75963741e-05, 1.44860315e-04, 2.41377496e-04, 3.39656502e-04]),
 array([5.85113108e-05, 1.75533932e-04, 2.92798773e-04, 4.09821638e-04]),
 array([6.79525624e-05, 2.04243665e-04, 3.40914855e-04, 4.77172262e-04]),
 array([7.80502317e-

In [ ]:
T_vals_new[:10]

[array([1.38556370e-06, 3.60045324e-06]),
 array([1.20649946e-05, 5.31006068e-05]),
 array([3.92645055e-05, 1.18974139e-04]),
 array([5.81202984e-05, 1.76419226e-04]),
 array([7.72343622e-05, 2.34052430e-04]),
 array([9.63693574e-05, 2.91635170e-04]),
 array([0.00011739, 0.00035094]),
 array([0.00013684, 0.00040868]),
 array([0.0001563 , 0.00046623]),
 array([0.00017572, 0.00052423])]

In [ ]:
s_select = 2
# n = np.linspace(4,32,9)
n = 4
piLength = 48e-9
data_points = 100
Tmax = np.linspace(5,700,data_points)*1e-6

w_extend = np.concatenate((w_in,w_low))
s_extend = np.concatenate((s_in[s_select,:],s_low))

lower_bounds = np.zeros(n)
for i in range (n):
  lower_bounds[i] = (1/2+i)*piLength

coeff_matrix = -np.diag(np.ones(n))
for i in range(n-1):
    coeff_matrix[i, i+1] = 1

M = coeff_matrix[:-1]
lower_B = piLength*np.ones(n-1)

C_vals_new = np.zeros(data_points)
C_vals = np.zeros(data_points)
T_vals = np.zeros((32,data_points))

for j in trange(data_points):
  upper_bounds = np.zeros(n)
  for i in range (n):
    upper_bounds[i] = Tmax[j]-(n-(2*i+1)/2)*piLength

  bounds  = Bounds(lower_bounds,upper_bounds)

  upper_B = (Tmax[j]-(n-1)*piLength)*np.ones(n-1)
  linear_constraint = LinearConstraint(M,lower_B,upper_B)

  x0 = cpmgFilter(n,Tmax[j])
  def customDD(x):
    c_val = -optCoherence(s_extend,w_extend,piLength,x,Tmax[j])
    return c_val

  res = minimize(customDD, x0, method='SLSQP',
               constraints=[linear_constraint], options={'ftol': 1e-9, 'disp': True},
               bounds=bounds)
  C_vals_new[j] = -customDD(res.x)
  C_vals[j] = -customDD(x0)
  T_vals[:n,j] = res.x


  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.9999831025243612
            Iterations: 5
            Function evaluations: 130
            Gradient evaluations: 3


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.9997034214740412
            Iterations: 15
            Function evaluations: 463
            Gradient evaluations: 11


...

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.03090454453939888
            Iterations: 73
            Function evaluations: 2622
            Gradient evaluations: 73


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.029807839949812873
            Iterations: 62
            Function evaluations: 2241
            Gradient evaluations: 62


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



Iteration limit reached    (Exit mode 9)
            Current function value: -0.028316067059009256
            Iterations: 100
            Function evaluations: 3518
            Gradient evaluations: 100


In [ ]:
C_vals_new[3,:].shape

(100,)

In [ ]:
fig = go.Figure()
fig.add_scatter(x=Tmax,y=C_vals_new, name = F'C_DD_{n}')
fig.add_scatter(x=Tmax,y=C_vals, name = F'C_CPMG_{n}')

# fig.add_scatter(x=Tmax,y=100*(C_vals_new/C_vals - 1), name = F'C_CPMG_{n}')

fig.update_layout(template=fig_template,  title = 'Filter Functions',
                  width = 1000,
                  # xaxis = dict(title='\N{greek small letter omega} (MHz)',range=[-6,1],type="log",),
                  xaxis = dict(title = 't'),
                  yaxis = dict(title='C(t)'),
                 )
fig.show()

In [ ]:
C_DD.shape

(2, 50)

In [ ]:
np.savez('/content/gdrive/MyDrive/C_vals_data_1.npz',C_vals_new_1 = C_vals_new,C_vals_1 = C_vals)


In [ ]:
load_data = np.load('/content/gdrive/MyDrive/C_vals_data.npz')

In [ ]:
np.geomspace(5,150)*1e-6

array([5.00000000e-06, 5.35938969e-06, 5.74461156e-06, 6.15752239e-06,
       6.60011239e-06, 7.07451486e-06, 7.58301639e-06, 8.12806797e-06,
       8.71229672e-06, 9.33851864e-06, 1.00097521e-05, 1.07292324e-05,
       1.15004275e-05, 1.23270545e-05, 1.32130978e-05, 1.41628280e-05,
       1.51808228e-05, 1.62719891e-05, 1.74415861e-05, 1.86952513e-05,
       2.00390274e-05, 2.14793913e-05, 2.30232857e-05, 2.46781520e-05,
       2.64519666e-05, 2.83532794e-05, 3.03912546e-05, 3.25757153e-05,
       3.49171905e-05, 3.74269662e-05, 4.01171393e-05, 4.30006765e-05,
       4.60914764e-05, 4.94044366e-05, 5.29555256e-05, 5.67618596e-05,
       6.08417849e-05, 6.52149669e-05, 6.99024842e-05, 7.49269306e-05,
       8.03125238e-05, 8.60852223e-05, 9.22728505e-05, 9.89052326e-05,
       1.06014337e-04, 1.13634429e-04, 1.21802237e-04, 1.30557130e-04,
       1.39941308e-04, 1.50000000e-04])

In [ ]:
C_CPMG.shape

(2, 50)

In [ ]:
load_data.files
C_vals_new = load_data['C_vals_new']
C_vals = load_data['C_vals']

In [ ]:
data_points = 20
Tmax = np.geomspace(5,150,data_points)*1e-6

In [ ]:
s_select = 15
n = 32
piLength = 48e-9
Tmax = 80e-6

w_extend = np.concatenate((w_in,w_low))
s_extend = np.concatenate((s_in[s_select,:],s_low))

def customDD(x):
  c_val = -optCoherence(s_extend,w_extend,piLength,x,Tmax)
  return c_val

In [ ]:
def customDD(x):
  c_val = -optCoherence(s_extend,w_extend,piLength,x,Tmax)
  return c_val

In [ ]:
lower_bounds = np.zeros(n)
for i in range (n):
  lower_bounds[i] = (1/2+i)*piLength
upper_bounds = np.zeros(n)
for i in range (n):
  upper_bounds[i] = Tmax-(n-(2*i+1)/2)*piLength

In [ ]:
coeff_matrix = -np.diag(np.ones(n))
for i in range(n-1):
    coeff_matrix[i, i+1] = 1

M = coeff_matrix[:-1]
lower_B = piLength*np.ones(n-1)
upper_B = (Tmax-(n-1)*piLength)*np.ones(n-1)

In [ ]:
# bounds = Bounds([piLength/2, 3*piLength/2], [Tmax-3*piLength/2, Tmax-piLength/2])
# linear_constraint = LinearConstraint([-1, 1], [piLength], [Tmax-2*piLength])

In [ ]:
bounds  = Bounds(lower_bounds,upper_bounds)
linear_constraint = LinearConstraint(M,lower_B,upper_B)

In [ ]:
x0 = cpmgFilter(n,Tmax)

In [ ]:
# x0 = cpmgFilter(n,Tmax)
# res = minimize(customDD, x0, method='trust-constr',
#                constraints=[linear_constraint],
#                options={'verbose': 1, 'maxiter': 1000, 'xtol': 1e-6}, bounds=bounds)

In [ ]:
x0 = cpmgFilter(n,Tmax)
res = minimize(customDD, x0, method='SLSQP',
               constraints=[linear_constraint], #options={'ftol': 1e-9, 'disp': True},
               bounds=bounds)

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



In [ ]:
from scipy import optimize
result = optimize.differential_evolution(customDD, bounds,constraints = linear_constraint)

KeyboardInterrupt: 

In [ ]:
result.fun

-0.5995761197663325

In [ ]:
results = optimize.shgo(customDD, bounds, constraints=linear_constraint)

In [ ]:
def eggholder(x):
    return (-(x[1] + 47) * np.sin(np.sqrt(abs(x[0]/2 + (x[1]  + 47))))
            -x[0] * np.sin(np.sqrt(abs(x[0] - (x[1]  + 47)))))

bounds = [(-512, 512), (-512, 512)]

In [ ]:
from scipy import optimize

results = optimize.shgo(eggholder, bounds)

In [ ]:
results['shgo_sobol'] = optimize.shgo(eggholder, bounds, n=200, iters=5,
                                      sampling_method='sobol')

In [ ]:
results.x

array([439.48097554, 453.97741501])

In [ ]:
eggholder()

-935.3379515604948

In [ ]:
x0,result.x

(array([1.250e-06, 3.750e-06, 6.250e-06, 8.750e-06, 1.125e-05, 1.375e-05,
        1.625e-05, 1.875e-05, 2.125e-05, 2.375e-05, 2.625e-05, 2.875e-05,
        3.125e-05, 3.375e-05, 3.625e-05, 3.875e-05, 4.125e-05, 4.375e-05,
        4.625e-05, 4.875e-05, 5.125e-05, 5.375e-05, 5.625e-05, 5.875e-05,
        6.125e-05, 6.375e-05, 6.625e-05, 6.875e-05, 7.125e-05, 7.375e-05,
        7.625e-05, 7.875e-05]),
 array([6.83732688e-08, 1.48525301e-07, 1.58739326e-06, 3.29747038e-06,
        4.14789921e-06, 4.27529971e-06, 5.84634455e-06, 8.71221697e-06,
        1.10709717e-05, 1.36273043e-05, 1.57232287e-05, 1.80683410e-05,
        2.03279357e-05, 2.29612666e-05, 2.51365462e-05, 2.75782188e-05,
        2.95893807e-05, 3.18001983e-05, 3.37076340e-05, 3.56920642e-05,
        3.76286110e-05, 3.97669098e-05, 4.18560316e-05, 4.39278189e-05,
        4.57677587e-05, 4.76685750e-05, 4.94498522e-05, 4.95197434e-05,
        4.96078883e-05, 5.18146143e-05, 5.40754230e-05, 6.57457617e-05]))

In [ ]:
customDD(x0)

-0.6300805508361996

In [ ]:
customDD(result.x)

-0.5995761197663325

In [ ]:
f_test = arbFilter(w_extend,piLength,cpmgFilter(n,Tmax),Tmax)
f_optimal = arbFilter(w_extend,piLength,result.x,Tmax)

fig = go.Figure()
fig.add_scatter(x=1e-6*w_extend/(2*np.pi),y=f_test)
fig.add_scatter(x=1e-6*w_extend/(2*np.pi),y=f_optimal)
fig.update_layout(template=fig_template,  title = 'Filter Functions',
                  width = 1000,
                  xaxis = dict(title='\N{greek small letter omega} (MHz)',range=[-6,1],type="log",),
                  yaxis = dict(title='F(\N{greek small letter omega}t)/\N{greek small letter omega}<sup>2</sup>'),
                 )
fig

In [ ]:
1/(w_in/(2*np.pi)).min()

0.006283185307179586

In [ ]:
w_in[-1]

3155.0046233746275

In [ ]:
w_low

array([3145.00462337, 3141.86961875, 3138.73461413, ...,   16.27000925,
         13.13500462,   10.        ])

### Train the network


In [ ]:
n_epochs = 200
start = time.time()
lstm, history_lstm = training(Net,x_train,y_train,batch=64,epochs=n_epochs,
                              lr=0.001,
                              )
end = time.time()
time_lstm = end - start
print(time_lstm)

Epoch 1/200
598/598 [==============================] - 25s 34ms/step - loss: 66.5779 - val_loss: 67.7152 - lr: 0.0010
Epoch 2/200
598/598 [==============================] - 20s 33ms/step - loss: 63.2474 - val_loss: 63.5890 - lr: 0.0010
Epoch 3/200
598/598 [==============================] - 19s 32ms/step - loss: 63.0457 - val_loss: 62.2173 - lr: 0.0010
Epoch 4/200
598/598 [==============================] - 19s 32ms/step - loss: 62.9528 - val_loss: 62.1639 - lr: 0.0010
Epoch 5/200
598/598 [==============================] - 19s 32ms/step - loss: 63.1725 - val_loss: 64.2890 - lr: 0.0010
Epoch 6/200
598/598 [==============================] - 19s 32ms/step - loss: 63.0421 - val_loss: 63.9276 - lr: 0.0010
Epoch 7/200
598/598 [==============================] - 19s 32ms/step - loss: 62.0848 - val_loss: 53.7998 - lr: 0.0010
Epoch 8/200
598/598 [==============================] - 19s 32ms/step - loss: 54.2960 - val_loss: 52.1369 - lr: 0.0010
Epoch 9/200
598/598 [==============================] - 1

In [ ]:
# Saving the trained model
lstm.save(trainData_path+'/LSTM_250_150_d200_150_d250_d501_ep200', overwrite=True)

# Saving the training history
pd.DataFrame(history_lstm.history).to_csv(path+'/LSTM_250_150_d200_150_d250_d501_ep200_history'+'.csv')

In [ ]:
y_predict = Net.predict(x_test)
err_predict = percent_error(y_test,y_predict,501)

235/235 [==============================] - 3s 9ms/step


In [ ]:
np.mean(err_predict)

9.235394769997226

In [ ]:
hist_lstm,x =np.histogram(err_predict,bins=51,range=[0,50],density=False)
x = (x[:-1]+x[1:])/2

In [ ]:
n_plot = 5
rand_set = np.random.randint(0,y_test.shape[0],(n_plot,))

In [ ]:
fig = make_subplots(rows=1, cols=3, subplot_titles=('Training History - LSTM','Mean Absolute Error', 'Comparison'))

fig.add_scatter(x=np.arange(1,301),y=history_lstm.history['val_loss'],line=dict(width=3,color='royalblue'),opacity=1,name=f'Validation loss',row=1,col=1,showlegend=False)
fig.add_trace(go.Bar(x=x,y=hist_lstm,opacity=0.75,name='Error',showlegend=False),row=1,col=2)
for i in range(n_plot):
  # fig.add_scatter(x=t*1e6,y=x_test[rand_set[i],:],line=dict(width=2,color=color_str[i]),opacity=1,name=f'Input-{rand_set[i]}',row=1,col=3)
  fig.add_scatter(x=1e-6*w_train/(2*np.pi),y=y_test[rand_set[i],:],line=dict(width=3,color=color_str[i]),opacity=0.8,name=f'Expected-{rand_set[i]}',showlegend=True,row=1,col=3)
  fig.add_scatter(x=1e-6*w_train/(2*np.pi),y=y_predict[rand_set[i],:],line=dict(width=3,color=color_str[i]),opacity=0.8,name=f'Predicted-{rand_set[i]}',showlegend=True,row=1,col=3)

fig.update_layout(template=fig_template,
                  width = 1000,
                  xaxis1 = dict(title='Epochs'),
                  yaxis1 = dict(title='Mean absolute error on validation set',type="log",),
                  xaxis2 = dict(title='% Error',range=[0,50]),
                  yaxis2 = dict(title='Samples'),
                  xaxis3 = dict(title='\N{greek small letter omega}/2\N{greek small letter pi} (MHz)',
                              #  type="log",
                              #  range=[,],
                               ),
                  yaxis3 = dict(title='S(\N{greek small letter omega})',
                              #  range=[-0.1e6,0.1e6],
                              #  type="log",

                               ),

                 )

In [ ]:
loaded_model = models.load_model(trainData_path+'/LSTM_250_150_d200_150_d250_d501_ep400')
# history_stored = pd.read_csv(path+'/cnn_trial'+'.csv')

### Predictions

In [ ]:
loaded_model0 = models.load_model(trainData_path+'/LSTM_250_150_d200_150_d250_d501_ep400')
loaded_model1 = models.load_model(trainData_path+'/LSTM_250_150_d200_150_d250_d501_ep200')

In [ ]:
# (0,1,2,9,10,11,12,13,14,18,19,20)
select_data = norm_data[(1,14),:]
exp_predict0 = loaded_model0.predict(select_data)
exp_predict1 = loaded_model1.predict(select_data)

1/1 [==============================] - 2s 2s/step


In [ ]:
## To extend the noise spectrum to lower frequencies

def spectrum_extend(exp_predict,w_train,w_new):
  w_lowSize = np.argwhere(w_new<w_train.min()).size
  w_lowArg = np.argwhere(w_new<w_train.min())[0]

  s_extend = np.zeros((exp_predict.shape[0],w_new.size))
  s_extend[:,int(w_lowArg):] = np.repeat(np.mean(exp_predict[:,-10:],axis=1),
                                        int(w_lowSize)).reshape(exp_predict.shape[0],int(w_lowSize))
  for i in range(exp_predict.shape[0]):
    s_extend[i,:int(w_lowArg)] = interpData(w_train,exp_predict[i,:],w_new[:int(w_lowArg)])
  return s_extend

## Calculate noise using delta-function approximation

def delta_noise(c, t):
  w_delta = 2*np.pi/t
  s_delta = -np.pi*np.log(c)/t
  return s_delta, w_delta

In [ ]:
s_delta, w_delta = delta_noise(select_data,T_train)

w_new = np.flipud(np.geomspace(4e3,w_train.max(),1001))

s_extend = spectrum_extend(exp_predict1,w_train,w_new)
c_predict = getCoherence(s_extend,w_new,T_train,1,48e-9)
c_delta = getCoherence(s_delta,w_delta,T_train,1,48e-9)

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:00<?, ?it/s]

In [ ]:
s_delta.shape

(2, 150)

In [ ]:
fig = go.Figure()
for j in range(select_data.shape[0]):
  fig.add_scatter(x=data['xval']*1e6,y=select_data[j,:],line=dict(width=4,color=color_str[j]),opacity=0.8,name=f'Expt. {j}')
  fig.add_scatter(x=T_train*1e6,y=c_predict[j,:],line=dict(width=3,color=color_str[j]),opacity=1,name=f'Predict. {j}')
  # fig.add_scatter(x=T_train*1e6,y=c_delta[j,:],line=dict(width=2),opacity=0.8,name=f'{j}')
fig.update_layout(template=fig_template,  title = 'Coherence Curves',
                  width = 600,
                  xaxis = dict(title='Evolution time (\N{greek small letter mu}s)',range=[0,400]),
                  yaxis = dict(title='Coherence'),
                 )

In [ ]:
fig = go.Figure()
for i in range(select_data.shape[0]):
  # fig.add_scatter(x=1e-6*w_train/(2*np.pi),y=exp_predict0[i,:],line=dict(width=3),opacity=0.8,showlegend=True,name=f'{i}')
  # fig.add_scatter(x=1e-6*w_train/(2*np.pi),y=exp_predict1[i,:],line=dict(width=3),opacity=0.8,showlegend=True,name=f'{i}')
  fig.add_scatter(x=1e-6*w_new/(2*np.pi),y=s_extend[i,:],line=dict(width=3),opacity=1,showlegend=True,name=f'Predict. {i}')

fig.add_scatter(x=1e-6*w_delta/(2*np.pi),y=s_delta[1,:],line=dict(width=3,color='grey'),opacity=0.2,showlegend=True,name=f'Approx.')

fig.update_layout(template=fig_template,
                  width = 600,
                  xaxis = dict(title='\N{greek small letter omega}/2\N{greek small letter pi} (MHz)',
                               type="log",
                              #  range=[,],
                               ),
                  yaxis = dict(title='S(\N{greek small letter omega})',
                               range=[3.6,4.6],
                               type="log",

                               ),

                 )

## CNN

In [ ]:
from tensorflow.keras import layers

In [ ]:
filter_nb=32
kernel_size=32
dropout_rate=0.03
pool_size=2
xtrain_size = np.shape(x_train)[-1]
ytrain_size = np.shape(y_train)[-1]

model = models.Sequential()
model.add( layers.Input( shape=(xtrain_size, 1) ) )
model.add( layers.Conv1D(filter_nb,kernel_size,activation="relu", padding='same' ) )
model.add( layers.Conv1D(filter_nb,kernel_size,activation="relu", padding='same' ) )
model.add( layers.MaxPooling1D( pool_size=pool_size, padding="same") )
model.add( layers.Conv1D(filter_nb/2, kernel_size,activation="relu", padding='same' ) )
model.add( layers.Flatten() )
model.add( layers.Dense(501, activation='relu') )
model.add( layers.Reshape((501,1) ))
# model.add( layers.MaxPooling1D( pool_size=pool_size, padding="same") )
model.add( layers.Conv1D(filter_nb,kernel_size,activation="relu", padding='same' ) )
# model.add( layers.MaxPooling1D( pool_size=pool_size, padding="same") )
# model.add( layers.Conv1D( filter_nb,kernel_size,activation="relu", padding='same' ) )
# model.add( layers.UpSampling1D( size=pool_size ) )
# model.add( layers.Conv1D( filter_nb,kernel_size,activation="relu", padding='same' ) )
# model.add( layers.UpSampling1D( size=pool_size ) )
model.add( layers.Conv1D( filter_nb,kernel_size,activation="relu", padding='same' ) )
# model.add( layers.UpSampling1D( size=pool_size ) )
# model.add( layers.Conv1D( filter_nb,kernel_size,activation="relu", padding='same' ) )
model.add( layers.Conv1D( 1, kernel_size, activation="relu", padding='same' ) )
model.add( layers.Flatten() )
# model.add( layers.Dense(250, activation='relu') )
model.add( layers.Dropout( dropout_rate ) )
model.add( layers.Dense(ytrain_size, activation='linear') )
model.compile(loss='MAPE',optimizer='adam')
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_62 (Conv1D)          (None, 150, 32)           1056      
                                                                 
 conv1d_63 (Conv1D)          (None, 150, 32)           32800     
                                                                 
 max_pooling1d_10 (MaxPooli  (None, 75, 32)            0         
 ng1D)                                                           
                                                                 
 conv1d_64 (Conv1D)          (None, 75, 16)            16400     
                                                                 
 flatten_16 (Flatten)        (None, 1200)              0         
                                                                 
 dense_27 (Dense)            (None, 501)               601701    
                                                     

In [ ]:
trainable_cnn = np.sum([K.count_params(w) for w in model.trainable_weights])
trainable_cnn

938340

In [ ]:
n_epochs = 450
start = time.time()
cnn, history_cnn = training(model,x_train,y_train,batch=64,epochs=n_epochs,
                              lr=0.001,
                              )
end = time.time()
time_cnn = end - start
print(time_cnn)

Epoch 1/450
120/120 [==============================] - 3s 10ms/step - loss: 82.9836 - val_loss: 64.6717 - lr: 0.0010
Epoch 2/450
120/120 [==============================] - 1s 7ms/step - loss: 62.2486 - val_loss: 57.0523 - lr: 0.0010
Epoch 3/450
120/120 [==============================] - 1s 7ms/step - loss: 56.2330 - val_loss: 51.9073 - lr: 0.0010
Epoch 4/450
120/120 [==============================] - 1s 7ms/step - loss: 55.5577 - val_loss: 53.4680 - lr: 0.0010
Epoch 5/450
120/120 [==============================] - 1s 8ms/step - loss: 55.6468 - val_loss: 53.4283 - lr: 0.0010
Epoch 6/450
120/120 [==============================] - 1s 8ms/step - loss: 54.4958 - val_loss: 54.3225 - lr: 0.0010
Epoch 7/450
120/120 [==============================] - 1s 7ms/step - loss: 54.0162 - val_loss: 50.6114 - lr: 0.0010
Epoch 8/450
120/120 [==============================] - 1s 7ms/step - loss: 52.7374 - val_loss: 51.2589 - lr: 0.0010
Epoch 9/450
120/120 [==============================] - 1s 7ms/step - lo

In [ ]:
# Saving the trained model
cnn.save(trainData_path+'/cnn_noUpsampling', overwrite=True)

# Saving the training history
pd.DataFrame(history_cnn.history).to_csv(path+'/cnn_noUpsampling'+'.csv')

In [ ]:
loaded_model = models.load_model(trainData_path+'/cnn_noUpsampling')
history_stored = pd.read_csv(path+'/cnn_noUpsampling'+'.csv')

In [ ]:
y_predict = model.predict(x_test)
err_predict = percent_error(y_test,y_predict,501)

47/47 [==============================] - 1s 7ms/step


In [ ]:
np.mean(err_predict)

11.518251752224373

In [ ]:
history_stored

,Unnamed: 0,loss,val_loss,lr
0,0,99.936836,99.762276,0.001000
1,1,81.220009,68.352989,0.001000
2,2,61.099972,54.433704,0.001000
3,3,57.290535,55.581230,0.001000
4,4,55.860741,53.377583,0.001000
...,...,...,...,...
229,229,13.527983,13.439760,0.000016
230,230,13.628733,13.031521,0.000016
231,231,13.656779,13.362166,0.000016
232,232,13.698814,13.390276,0.000016


In [ ]:
hist_cnn,x =np.histogram(err_predict,bins=51,range=[0,50],density=False)
x = (x[:-1]+x[1:])/2

In [ ]:
n_plot = 5
rand_set = np.random.randint(0,y_test.shape[0],(n_plot,))

In [ ]:
fig = make_subplots(rows=1, cols=3, subplot_titles=('Training History - CNN','Mean Absolute Error', 'Comparison'))
# history_cnn.history['val_loss']
fig.add_scatter(x=np.arange(1,301),y=history_cnn.history['val_loss'],line=dict(width=3,color='royalblue'),opacity=1,name=f'Validation loss',row=1,col=1,showlegend=False)
fig.add_trace(go.Bar(x=x,y=hist_cnn,opacity=0.75,name='Error',showlegend=False),row=1,col=2)
for i in range(n_plot):
  # fig.add_scatter(x=t*1e6,y=x_test[rand_set[i],:],line=dict(width=2,color=color_str[i]),opacity=1,name=f'Input-{rand_set[i]}',row=1,col=3)
  fig.add_scatter(x=1e-6*w_train/(2*np.pi),y=y_test[rand_set[i],:],line=dict(width=3,color=color_str[i]),opacity=0.8,name=f'Expected-{rand_set[i]}',showlegend=True,row=1,col=3)
  fig.add_scatter(x=1e-6*w_train/(2*np.pi),y=y_predict[rand_set[i],:],line=dict(width=3,color=color_str[i]),opacity=0.8,name=f'Predicted-{rand_set[i]}',showlegend=True,row=1,col=3)

fig.update_layout(template=fig_template,
                  width = 1000,
                  xaxis1 = dict(title='Epochs'),
                  yaxis1 = dict(title='Mean absolute error on validation set',type="log",),
                  xaxis2 = dict(title='% Error',range=[0,50]),
                  yaxis2 = dict(title='Samples'),
                  xaxis3 = dict(title='\N{greek small letter omega}/2\N{greek small letter pi} (MHz)',
                              #  type="log",
                              #  range=[,],
                               ),
                  yaxis3 = dict(title='S(\N{greek small letter omega})',
                              #  range=[-0.1e6,0.1e6],
                              #  type="log",

                               ),

                 )